# Obtención y preprocesamiento de data

Para obtener la data, se tiene acceso a ensembl (http://plants.ensembl.org/prot_tree_stats.html) y cantatadb (http://cantata.amu.edu.pl).

El modelo a entrenar, necesitará información balanceada de distintas especies. Esto quiere decir que todas las especies deben tener la misma cantidad de ARNs codificantes y de ARNs no codificantes. Por lo tanto, será necesario obtener la cantidad de ARN codificantes y ARN no codificantes, para decidir las especies que serán utilizadas para el modelo. 

Con respecto a la información en ensembl, se accedió a la misma a través de biomaRt, que es un paquete para el lenguaje R, mediante el cuál se puede interactuar con la base de datos de transcriptomas.

De ensembl (plantas), las 10 especies con la mayor cantidad de información (ARNs codificantes) son:

In [1]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
df = pd.read_sql_query("SELECT especie, conteo_pct FROM conteo_lncrna_especie ORDER BY conteo_pct desc LIMIT 10", conn)
display(df)

conn.close()

,especie,conteo_pct
0,Triticum aestivum,107545
1,Brassica napus,101040
2,Triticum dicoccoides\r\n,62569
3,Brassica oleracea,59220
4,Glycine max,55897
5,Helianthus annuus,52191
6,Medicago truncatula,50444
7,Populus trichocarpa,41335
8,Brassica rapa,41018
9,Oryza sativa Indica Group,40745


Con respecto a la información de cantatadb, se descagaron desde la web los archivos en formatos fasta y se formatearon.

De cantatadb, las 10 especies con la mayor cantidad de información (lncRNA) son:

In [2]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
df = pd.read_sql_query("SELECT especie, conteo_lncRNA FROM conteo_lncrna_especie ORDER BY conteo_lncRNA desc LIMIT 10", conn)
display(df)

conn.close()

,especie,conteo_lncRNA
0,Chenopodium_quinoa,17526
1,Brassica_napus,12010
2,Malus_domestica,10924
3,Ananas_comosus,10404
4,Oryza_rufipogon,10261
5,Trifolium_pratense,10179
6,Manihot_esculenta,9504
7,Oryza_nivara,8955
8,Brassica_rapa,8501
9,Hordeum_vulgare,7970


Para tomar la decisión de qué especies utilizar, conviene seleccionar aquellas con la mayor cantidad de información. Debido a que claramente siempre existen más ARN codificantes que no codificantes, se utilizarán los ARN no codificantes como criterio de ordenamiento. Adicionalmente se añadirá la condición de que existan transcriptomas de ambos tipo para una misma especie.

In [3]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
df = pd.read_sql_query("SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA FROM conteo_lncrna_especie c1, conteo_lncrna_especie c2 WHERE c1.especie = REPLACE(c2.especie, '_', ' ') ORDER BY 3 DESC, 2 DESC LIMIT 10", conn)
display(df)

conn.close()

,especie,conteo_pct,conteo_lncRNA
0,Brassica napus,101040,12010
1,Oryza rufipogon,37071,10261
2,Trifolium pratense,39917,10179
3,Manihot esculenta,33044,9504
4,Oryza nivara,36313,8955
5,Brassica rapa,41018,8501
6,Hordeum vulgare,37705,7970
7,Arabidopsis lyrata,32667,7593
8,Cucumis sativus,23780,7348
9,Brassica oleracea,59220,7338


El único problema, es que la información de cantatadb es de lncRNA, y originalmente se deseaba trabajar con lincRNA. Los lincRNA son aquellos lncRNA que no se superponen con secciones codificantes.